# **Predicting Rankings of Tennis Players: The 2018 ATP Tour**

<br>

# **Introduction**

By using the data set of games won or lost by a tennis player, we are able to use several categorical variables to predict and classify the future ranking of a particular tennis player based on these attributes. This leads to the question, how well do variables relating to player status, age, current rank, and playing hand predict, and game statistics the future ranking of the player during later tennis seasons? The dataset used to answer this question will be the game results for the top 500 players during the 2018 tennis season. This particular data was chosen to avoid interruptions by the COVID-19 pandemic and to ensure that the data would be stagnant, with no new data incoming. The dataset includes data from both winners and losers of rounds of national and international tennis tournaments, hosted by the Association of Tennis Professionals (ATP).

<br>

# **Preliminary Exploratory Data Analysis**

### **Importing Libraries**

In [12]:
library(tidyverse)
library(tidymodels)
library(repr)
library(dplyr)

options(repr.matrix.max.rows = 6)

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3      ✔ yardstick 0.0.7 

Warning message:
“package ‘broom’ was built under R version 4.0.2”
Warning message:
“package ‘dials’ was built under R version 4.0.2”
Warning message:
“package ‘infer’ was built under R version 4.0.3”
Warning message:
“package ‘modeldata’ was built under R version 4.0.1”
Warning message:
“package ‘parsnip’ was built under R version 4.0.2”
Warning message:
“package ‘recipes’ was built under R version 4.0.1”
Warning message:
“package ‘tune’ was built under R version 4.0.2”
Warning message:
“package ‘workflows’ was built under R version 4.0.2”
Warning message:
“package ‘yardstick’ was built under R version 4.0.2”
── Conflicts ────

### **Download the Tennis Dataset**

In [2]:
url <- "https://drive.google.com/uc?export=download&id=1fOQ8sy_qMkQiQEAO6uFdRX4tLI8EpSTn"
download.file(url, "data/data.csv")

### **Load the Tennis Dataset**

In [6]:
tennis <- read_csv("data/data.csv")
head(tennis)

Warning message:
“Missing column names filled in: 'X1' [1]”
Parsed with column specification:
cols(
  .default = col_double(),
  tourney_id = col_character(),
  tourney_name = col_character(),
  surface = col_character(),
  tourney_level = col_character(),
  winner_seed = col_character(),
  winner_entry = col_character(),
  winner_name = col_character(),
  winner_hand = col_character(),
  winner_ioc = col_character(),
  loser_seed = col_character(),
  loser_entry = col_character(),
  loser_name = col_character(),
  loser_hand = col_character(),
  loser_ioc = col_character(),
  score = col_character(),
  round = col_character()
)

See spec(...) for full column specifications.



X1,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,⋯,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,2019-M020,Brisbane,Hard,32,A,20181231,300,105453,2,⋯,54,34,20,14,10,15,9,3590,16,1977
1,2019-M020,Brisbane,Hard,32,A,20181231,299,106421,4,⋯,52,36,7,10,10,13,16,1977,239,200
2,2019-M020,Brisbane,Hard,32,A,20181231,298,105453,2,⋯,27,15,6,8,1,5,9,3590,40,1050
3,2019-M020,Brisbane,Hard,32,A,20181231,297,104542,NA,⋯,60,38,9,11,4,6,239,200,31,1298
4,2019-M020,Brisbane,Hard,32,A,20181231,296,106421,4,⋯,56,46,19,15,2,4,16,1977,18,1855
5,2019-M020,Brisbane,Hard,32,A,20181231,295,104871,NA,⋯,54,40,18,15,6,9,40,1050,185,275


### **Filtering the Dataset**

In [7]:
tennis <- subset(tennis, select = c(tourney_name, draw_size, tourney_level, tourney_date, winner_name, winner_hand, winner_ht, winner_ioc, winner_age, score, best_of, w_ace, w_df, w_svpt, w_1stIn, w_1stWon, w_2ndWon, w_SvGms, w_bpSaved, w_bpFaced, winner_rank, winner_rank_points))

In [9]:
tennis <- filter(tennis, substr(tourney_date, 0, 4) == "2018")
tennis <- mutate(tennis, winner_hand = as.integer(as.factor(winner_hand)), na.rm = TRUE)
tennis

tourney_name,draw_size,tourney_level,tourney_date,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,score,⋯,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,winner_rank,winner_rank_points,na.rm
<chr>,<dbl>,<chr>,<dbl>,<chr>,<int>,<dbl>,<chr>,<dbl>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
Brisbane,32,A,20181231,Kei Nishikori,2,178,JPN,29.00479,6-4 3-6 6-2,⋯,77,44,31,17,13,3,6,9,3590,TRUE
Brisbane,32,A,20181231,Daniil Medvedev,2,NA,RUS,22.88569,7-6(6) 6-2,⋯,52,33,28,14,10,0,1,16,1977,TRUE
Brisbane,32,A,20181231,Kei Nishikori,2,178,JPN,29.00479,6-2 6-2,⋯,47,33,26,9,8,2,2,9,3590,TRUE
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Davis Cup WG F: FRA vs CRO,4,D,20181123,Borna Coric,2,NA,CRO,22.01232,6-2 7-5 6-4,⋯,78,59,47,15,15,2,2,12,2480,TRUE
Davis Cup WG F: FRA vs CRO,4,D,20181123,Marin Cilic,2,198,CRO,30.14100,6-3 7-5 6-4,⋯,86,53,48,20,16,5,5,7,4250,TRUE
Davis Cup WG F: FRA vs CRO,4,D,20181123,Marin Cilic,2,198,CRO,30.14100,7-6(3) 6-3 6-3,⋯,83,44,39,26,15,0,0,7,4250,TRUE


#### **Group By Player Name**

In [10]:
player_stats <- group_by(tennis, winner_name) %>% 
    summarize(winner_rank = mean(winner_rank), winner_age = median(winner_age), winner_hand = mean(winner_hand), w_ace = mean(w_ace), w_df = mean(w_df), w_svpt = mean(w_svpt), w_1stIn = mean(w_1stIn), w_1stWon = mean(w_1stWon), w_2ndWon = mean(w_2ndWon), w_SvGms = mean(w_SvGms), w_bpSaved = mean(w_bpSaved), w_bpFaced = mean(w_bpFaced)) %>% 
    na.omit(player_stats)

player_stats

`summarise()` ungrouping output (override with `.groups` argument)



winner_name,winner_rank,winner_age,winner_hand,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Adam Pavlasek,196.50,23.55921,2,10.00,3.00,101.0,72.50,52.50,17.50,16.50,4.00,5.00
Adrian Mannarino,30.24,29.98768,1,6.76,2.56,86.4,53.96,40.12,18.68,14.08,3.68,5.52
Adrian Menendez Maceiras,128.00,32.29295,2,7.50,1.50,89.0,63.00,41.00,15.50,13.00,6.00,8.50
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Yuichi Sugita,46.71429,29.72758,2,5.571429,3,82.71429,51.42857,40,16.57143,13.28571,4.142857,5.714286
Ze Zhang,247.00000,27.74538,2,4.000000,4,54.00000,35.00000,25,10.00000,9.00000,4.000000,6.000000
Zsombor Piros,419.00000,18.91034,3,12.000000,2,159.00000,102.00000,73,30.00000,27.00000,7.000000,12.000000


### **Splitting the Data into Training and Testing Set**

In [14]:
set.seed(2000)
player_split <- initial_split(player_stats, prop = 3/4, strata = winner_rank)
player_training <- training(player_split)
player_testing <- testing(player_split)

player_testing
player_training

winner_name,winner_rank,winner_age,winner_hand,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Adam Pavlasek,196.5,23.55921,2,10.000000,3.000000,101,72.5,52.5,17.50000,16.5,4.000000,5.000000
Alejandro Gonzalez,328.0,28.97467,2,3.000000,2.000000,66,44.0,29.0,11.00000,11.0,4.000000,7.000000
Andrej Martin,156.0,28.37782,2,3.666667,2.666667,63,38.0,28.0,13.66667,10.0,5.666667,7.333333
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Viktor Galovic,267.0,27.84120,2,5.0,4.0,90.5,54.0,38.5,18.0,13.5,3.5,5.5
Yoshihito Nishioka,206.1,22.97331,1,1.2,1.8,80.0,56.5,37.6,13.1,12.6,3.9,6.7
Yosuke Watanuki,273.0,20.47091,3,10.0,2.0,78.0,44.0,33.0,22.0,14.0,0.0,3.0


winner_name,winner_rank,winner_age,winner_hand,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Adrian Mannarino,30.240,29.98768,1,6.7600,2.56,86.4,53.960,40.1200,18.68,14.080,3.6800,5.520
Adrian Menendez Maceiras,128.000,32.29295,2,7.5000,1.50,89.0,63.000,41.0000,15.50,13.000,6.0000,8.500
Albert Ramos,33.875,30.28337,1,4.9375,3.25,92.5,58.125,42.8125,19.00,14.875,3.6875,6.125
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Yuichi Sugita,46.71429,29.72758,2,5.571429,3,82.71429,51.42857,40,16.57143,13.28571,4.142857,5.714286
Ze Zhang,247.00000,27.74538,2,4.000000,4,54.00000,35.00000,25,10.00000,9.00000,4.000000,6.000000
Zsombor Piros,419.00000,18.91034,3,12.000000,2,159.00000,102.00000,73,30.00000,27.00000,7.000000,12.000000


In [ ]:
# ggplot(player_training, aes(x = , y = ))

<br>

# **Methods**

From the data set, we will be using only the columns of rank, age, playing hand, and several other game statistics to predict the future ranks of tennis players. Each of these variables was averaged to ensure that data throughout the ATP circuit would be included in the analysis. We will be using regression analysis that uses data to understand how a ranking variable's value is affected when one or more independent variables change or stay the same. By understanding each variable's relationship and how they developed in the past, we will anticipate possible outcomes and make better predictions for future ranking. Due to the amount of the data in the orignal dataset, many of the tennis players were not included in this analysis due to a lack of, or insufficient data. These players were filtered out during the intial phase of exploring our data. Furthermore, only the data based on the winners of each tournament round were considered. This is due to the fact that the points that determine a player's ranking are only awarded when matches are won, and are never deducted for losing. Losing has no impact on the ranking of the player, our predicted variable.

We will visualize the results by using graphs to find the relationship between the data. We will be using scatterplot and functions such as ggplot/ggpairs to visualize the results with future ranking on the y-axis and rest on the x-axis. 


<br>

# **Expected Outcomes and Significance**

   It is to be expected that tennis player statistics, to a certain degree, should be able to predict their current and future rankings. In general observation, the majority of the top players will remain near the top of the rankings for the majority of their careers. It will be interesting to see if any predictors can significantly predict which players are to rise or fall in their rankings. It is expected that age, current rankings, and their previous rise in rankings over preceding years. There are also some factors that will be interesting to explore their effect on world rankings, such as physical attributes. 
   
   It’ll be interesting to explore whether, and if so, the extent, the future ranking of Tennis players can be forecasted using current statistics. Although there are not any significant real-world impacts that any of our findings will lead to, it will be an interesting read for any sports fans and data junkies to read and expand upon. Based on our findings, it will be interesting to explore whether there are any missing factors that could improve on our prediction model; and the result of applying our model to other sports as well.

